In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Dense,BatchNormalization,Input,Bidirectional,Conv1D,MaxPooling1D,Flatten
from tensorflow.keras import regularizers
import tensorflow as tf
import numpy as np

In [ ]:
df = []
for i in range(0,5):
    df.append(pd.read_csv(f'history_log_{i}.csv'))

In [ ]:
model = Sequential([
        LSTM(50, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), input_shape=(256,3), return_sequences = False ),
        Dropout(0.2),
        Dense(2, activation = 'softmax') 
        ])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
for i in range(len(df)):
    plot = df[i][:].reset_index()[['loss','accuracy','val_accuracy']].plot(title = f'{i+1} Fold',xlabel = 'epoch',ylabel = 'accuracy and loss')
    fig = plot.get_figure()
    fig.savefig(f'{i}_fold.png')    

In [ ]:
from IPython.display import display
for i in range(len(df)):
    display(df[i][df[i]['epoch']==0])

In [ ]:
from PIL import ImageFont
import visualkeras
font = ImageFont.truetype("arial.ttf", 32)  # using comic sans is strictly prohibited!
visualkeras.layered_view(model, legend=True, font=font,draw_volume=False)  # font is optional!

In [ ]:
table = []
for i in range(len(df)):
    display(df[i][df[i]['val_accuracy'] == df[i]['val_accuracy'].max()][:1])
    table.append(df[i][df[i]['val_accuracy'] == df[i]['val_accuracy'].max()][:1])
table = pd.concat(table)

In [ ]:
import pickle

# open a file, where you stored the pickled data
file = open('scores', 'rb')
# dump information to that file
scores = pickle.load(file)
# close the file
file.close()

# open a file, where you stored the pickled data
file = open('evaluations', 'rb')
# dump information to that file
evaluations = pickle.load(file)
# close the file
file.close()


In [ ]:
scores = np.asarray(scores).T

In [ ]:
scores

In [ ]:
df3 = pd.DataFrame(scores,columns = ['f1','recalls','precisions'])

In [ ]:
df3

In [ ]:
np.asarray(evaluations)[:,1]

In [ ]:
df3

In [ ]:
df2 = table[['accuracy','val_accuracy','epoch']].reset_index()
df2 = pd.concat([df2,df3],axis = 1)
df2['test_accuracy'] = np.asarray(evaluations)[:,1]


In [ ]:
s = pd.DataFrame([df2.sum()])/5

In [ ]:
df2 = pd.concat([df2,s])

In [ ]:
df2 = ((df2*10000)//1)/10000

In [ ]:
df2.columns

In [ ]:
df2 = df2[['accuracy', 'val_accuracy','test_accuracy','epoch','recalls','precisions','f1']]

In [ ]:
df2

In [ ]:
df2.to_csv('table.csv')

In [ ]:
df2